In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# html을 이용하여 가로방향으로 여러 DF을 인쇄
class disp(object):
    template = '<div style="float: left;padding:10px;"> <b>[{0}]</b> {1}</div>'
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args )

import pandas as pd   
import numpy as np
import seaborn as sns

### [예제1] GroupBy 집계1

In [ ]:
df = pd.DataFrame({'brand':list('AABAB'), 
                  'model':['TV', 'PC', 'PC', 'TV', 'TV'], 
                   'sale':[10,30,50,40,70], 'stock':[100,100,200,200,300]})
df                  

In [ ]:
# [1]

gb1 = df.groupby('model')
r1 = gb1.count()
r2 = gb1['model'].count()
r3 = gb1.size()     

r1; r2; r3

In [ ]:
# [2]

gb1 = df.groupby('model')
r4 = gb1.sum()#.sum(axis=1)
r5 = gb1['brand'].value_counts()

df; r4; r5

In [ ]:
# [3]

gb2 = df.groupby(['model','brand'])
r6 = gb2.sum().query('brand=="A" and sale<=50')
r7 = gb2.sum().groupby(level=1).sum()

gb2.sum(); r6; r7

### [예제2] 그룹집계 활용 예

In [ ]:
df = pd.DataFrame({'month':[3,3,9,9,9], 'brand':list('AABAB'), 
                  'model':['TV', 'PC', 'PC', 'TV', 'TV'], 
                   'sale':[90,30,50,40,70]})
df                  

In [ ]:
# [1]

func=lambda x:'high' if df.loc[x,'sale']>=50 else 'low'
sr1 = df.groupby(['month', func])['model'].count()
r1 = sr1.unstack(fill_value=0)

sr1; r1

In [ ]:
# [2]

month_sum = df.groupby('month').sum()
model_sum=df.groupby(['month','model']).sum()
r2 = (model_sum / month_sum * 100).round()

disp('month_sum', 'model_sum', 'r2')

### [예제3] apply(), agg(), transform() 메서드 비교

In [ ]:
df = pd.DataFrame({'grade':[1,1,1,2,2],'names':['kim', 'lee', 'park','song','lew'],
                  'kor':[100,80,60,90,80], 'eng':[60,70,80,100,70]})
df

In [ ]:
# [1]

def func(x):
    return pd.Series({'eng': x['eng'].max(),'kor':x['kor'].min()})

r1 = df.pivot_table(index='grade',aggfunc={'eng':max,'kor':min})
r2 = df.groupby('grade').agg({'eng':max, 'kor':min})
r3 = df.groupby('grade').apply(func)

r1; r2; r3

In [ ]:
# [2]

def func(x):
    return x.max() - x.min()
r4 = df.pivot_table(index='grade', aggfunc=func)
r5 = df.groupby('grade').agg(func)
r6 = df.groupby('grade')[['kor','eng']].apply(func)
r7 = df.groupby('grade').transform(func)

r4; r5; r6; r7

### [예제4] apply(), agg(), transform() 메서드의 이해

In [ ]:
df = pd.DataFrame({'grade':[1,1,1,2,2],'names':['kim', 'lee', 'park','song','lew'],
                  'kor':[100,80,60,90,80], 'eng':[60,70,80,100,70]})
df

In [ ]:
# [1-1]  

def func(x):
    print(x)

gb1= df.groupby('grade')
r1 = gb1.apply(func)
r1

In [ ]:
# [1-2]  

def func(x):
    print(x)

gb1= df.groupby('grade')
r2 = gb1.agg(func)
r2

In [ ]:
# [1-3]  

def func(x):
    print(x)

gb1= df.groupby('grade')
r3 = gb1.transform(func)
r3

### [예제5] GroupBy.agg() 메서드의 이해

In [ ]:
df = pd.DataFrame({'grade':[1,2,1,2,1],'names':['kim', 'lee', 'park','song','lew'],
                  'kor':[100,80,60,90,80], 'eng':[60,50,80,100,70], 
                  'absent':[3, 0, 1, 5, 2]})

In [ ]:
# [1-1] 

gb1 = df.groupby('grade')[['kor', 'eng']]
r1 = gb1.agg('mean')
r2 = gb1.agg(['sum', 'mean'])
r3 = gb1.agg({'kor':'sum', 'eng':'mean'})
disp('df','r1'); disp('r2','r3')

In [ ]:
# [1-2] 

gb1 = df.groupby('grade')[['kor', 'eng']]
r4 = gb1.agg([('tot','sum'), ('avg','mean')])
r5 = gb1.agg(tot=('kor','sum'))
disp('df', 'r4','r5')

In [ ]:
# [2]

def func(x):
    return x.max() - x.min()

gb2 = df.groupby('grade')['absent']
r6 = gb2.agg([func])
r7 = gb2.agg([all, any])
disp('df', 'r6', 'r7')

### [예제6] GroupBy.filter() 메서드의 이해

In [ ]:
df = pd.DataFrame({'grade':[1,1,2,2,3,3],
                   'names':['kim','lee','park','song','lew','ki'],
                  'exam':[90,80,50,70,80,60]})
df

In [ ]:
# [1-1]

gb = df.groupby('grade')
r1 = gb.agg('mean')
disp('df','r1')

In [ ]:
# [1-2]

gb = df.groupby('grade')
func = lambda x : x['exam'].mean()>=70
r2 = gb.filter(func)
r3 = gb.filter(func, dropna=False)

disp('df', 'r2', 'r3')

### [예제7] 그룹집계 활용 : titanic

In [ ]:
df = sns.load_dataset('titanic')[['pclass','survived', 'sex', 'age']]
#df = pd.read_csv('data/titanic.csv',usecols=['pclass','survived','sex','age']) 
df.head()

In [ ]:
# [1]

gb = df.groupby(['pclass', 'sex'])
df['age'] = df['age'].fillna(gb['age'].transform('median'))

df.info()

In [ ]:
# [2] 

df['age2'] = df['age'].apply(lambda x: x//10*10)
gb = df.groupby(['pclass', 'age2'])
r1 = gb['survived'].sum()
r2 = gb.filter(lambda x: (x['survived'].sum() >=50) )

r1; r2.head()